# Strava segment data

Choose some activity ID or the LAST one if `LAST=True`

In [1]:
activity=4439026712#4310029237#4088494919
date='December 6, 2020'
LAST=True

In [2]:
import helium
import pandas as pd
import os
import time
import sys
def metadata(**kwargs):
    return kwargs
db_dir='db'
db_file='{}/strava.json'.format(db_dir)

In [3]:
bkp=pd.read_json(db_file)
if not LAST and not bkp[bkp.Info.astype(str).str.contains(date)].empty:
    sys.exit('Check if already exists in bkp')

In [4]:
os.makedirs(db_dir,exist_ok=True)

if os.path.isfile(db_file):
    df=pd.read_json(db_file)
else:
    df=pd.DataFrame()

browser=helium.start_chrome("https://www.strava.com/login")

helium.write('diego.restrepo@gmail.com',into='Your Email')

helium.write('opoqdh11',into='Password')

helium.click('Log In')

time.sleep(10)
browser.maximize_window()

if LAST:
    #Required to activate card-section below
    lt=[t for t in browser.find_elements_by_class_name('card-section') 
       if t.text.find('Latest Activity')>-1][0]

    lt.find_element_by_tag_name('a').click()

else:
    browser.get('https://www.strava.com/activities/{}'.format(activity))
# Web page with latest activity

# General info

time.sleep(10)
dt=browser.find_element_by_class_name('details')

date=dt.find_element_by_tag_name('time').text

name=dt.find_element_by_tag_name('h1').text

kk=browser.find_elements_by_tag_name("ul")

q=[ k for k in kk if k.text.find('\nDistance')>-1][0]

distance,time,elevation=(t.text for t in q.find_elements_by_tag_name('strong') )

q=[ k for k in kk if k.text.find('\nEstimated Avg Power')>-1][0]

power,energy=(t.text for t in q.find_elements_by_tag_name('strong') )

# Load tables

HIDDEN=True
try:
    browser.find_element_by_id('show-hidden-efforts').click()
except:
    HIDDEN=False
    pass

html=browser.page_source

ll=pd.read_html(html)

# General info table

mtdt=ll[0].groupby('Unnamed: 0').sum().transpose().to_dict()

mtdt.update(  metadata(date=date,name=name,distance=distance,time=time,
         elevation=elevation,power=elevation,energy=energy)
   )

#Fix time
import time

# Segments

sgmt=ll[1][['Name','Time','Speed','Power']].reset_index(drop=True)

if HIDDEN:
    sgmt=sgmt.append( ll[2].rename({2:'Name',4:'Time',5:'Speed',6:'Power'},
            axis='columns')[['Name','Time','Speed','Power']] 
           ).reset_index(drop=True)

#Add general info
sgmt['Info']=''

sgmt['Info'] = [ mtdt  for i in sgmt.index]

In [5]:
ll[0]

,Unnamed: 0,Avg,Max
0,Speed,20.5km/h,83.5km/h
1,Calories,1580,1580
2,Elapsed Time,3:22:45,3:22:45


In [6]:
sgmt

,Name,Time,Speed,Power,Info
0,Bomba Terpel - Semáforo Cll 69sur 0.97km 16m...,8:21,7.0km/h,93W,"{'Calories': {'Avg': '1580', 'Max': '1580'}, '..."
1,Semaforo zona Norte 2 - Semaforo cruce 0.37km ...,6:21,3.5km/h,58W,"{'Calories': {'Avg': '1580', 'Max': '1580'}, '..."
2,Puente 77 Sur - Variante 0.75km 29m 3.8%,3:16,13.8km/h,214W,"{'Calories': {'Avg': '1580', 'Max': '1580'}, '..."
3,puete la estrella - partidas 11.18km 218m 1.8%,39:36,16.9km/h,220W,"{'Calories': {'Avg': '1580', 'Max': '1580'}, '..."
4,Union - Aries 0.54km 26m 4.4%,2:23,13.7km/h,247W,"{'Calories': {'Avg': '1580', 'Max': '1580'}, '..."
5,Unicornio - Monaco 2.28km 85m 3.1%,8:04,17.0km/h,243W,"{'Calories': {'Avg': '1580', 'Max': '1580'}, '..."
6,unicornio ancon 1.13km 63m 5.4%,4:23,15.6km/h,252W,"{'Calories': {'Avg': '1580', 'Max': '1580'}, '..."
7,Sprint Largo (El ciclista) 0.61km 22m 3.3%,2:26,15.2km/h,205W,"{'Calories': {'Avg': '1580', 'Max': '1580'}, '..."
8,Sprint (El ciclista) 0.34km 17m 5.1%,1:12,17.0km/h,292W,"{'Calories': {'Avg': '1580', 'Max': '1580'}, '..."
9,Ancón Sur - Caldas 6.22km 88m 1.4%,20:16,18.4km/h,218W,"{'Calories': {'Avg': '1580', 'Max': '1580'}, '..."


In [7]:
df=df.append(sgmt,sort=False).reset_index(drop=True)

In [8]:
df['istr']=df['Info'].astype(str)

In [9]:
df=df.drop_duplicates(['Name', 'Power', 'Speed', 'Time', 'istr']).drop('istr',axis='columns').reset_index(drop=True)

In [10]:
df.to_json(db_file)

### Save notebook here

In [11]:
f=open('commit.sh','w')
f.write('git commit -am "{}"\n'.format(
    df.iloc[-1]['Info']['name']+' '+df.iloc[-1]['Info']['date']
))
f.write('git push origin master')
f.close()

In [12]:
!bash commit.sh

[master e840689] Morning Ride 7:01 AM on Monday, March 22, 2021
 2 files changed, 526 insertions(+), 837 deletions(-)
 rewrite Strava.ipynb (70%)
Enumerating objects: 9, done.
Counting objects: 100% (9/9), done.
Delta compression using up to 8 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (5/5), 2.91 KiB | 156.00 KiB/s, done.
Total 5 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To github.com:restrepo/strava.git
   4af767d..e840689  master -> master


In [13]:
if not( input('Close browser?').find('n')>-1 ):
    browser.close()

Close browser? 


WebDriverException: Message: chrome not reachable
  (Session info: chrome=88.0.4324.182)
  (Driver info: chromedriver=2.38.552522 (437e6fbedfa8762dec75e2c5b3ddb86763dc9dcb),platform=Linux 4.19.0-10-amd64 x86_64)
